In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import os
import copy
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler
from PIL import Image

C:\Users\snail\anaconda3\envs\3Dvision\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\snail\anaconda3\envs\3Dvision\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# check cuda
torch.cuda.is_available()

True

In [3]:
# use cuda
device = torch.device('cuda:0')

Get Dataset

In [9]:
train_triplets = open('train_triplets.txt','r')

In [11]:
list_anchor = []
list_positive = []
list_negative =[]

In [12]:
for line in train_triplets.readlines():
    image_id = line.strip().split(' ')
    list_anchor.append(['./food/'+str(image_id[0])+'.jpg','anchor'])
    list_positive.append(['./food/'+str(image_id[1])+'.jpg','positive'])
    list_negative.append(['./food/'+str(image_id[-1])+'.jpg','negative'])

In [13]:
list_anchor[0][0]

'./food/02461.jpg'

In [14]:
len(list_anchor)

59515

In [15]:
trainlist_anchor=list_anchor[:30000]
trainlist_positive=list_positive[:30000]
trainlist_negative=list_negative[:30000]

validlist_anchor=list_anchor[30000:]
validlist_positive=list_positive[30000:]
validlist_negative=list_negative[30000:]

In [16]:
class Mydata(Dataset):
    
    def __init__(self, anchor_pathlist, positive_pathlist, negative_pathlist,transform_type='valid'):
        self.anchor_pathlist = anchor_pathlist
        self.positive_pathlist = positive_pathlist
        self.negative_pathlist = negative_pathlist
        self.transform_type = transform_type
        if transform_type == 'train':
            self.transform_type = torchvision.transforms.Compose([transforms.Resize((224,224)),
                                                             transforms.RandomHorizontalFlip(p=0.5),
                                                             transforms.RandomVerticalFlip(p=0.5),
                                                             transforms.RandomRotation(degrees=90),
                                                             transforms.ToTensor(),
                                                             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                            ])
        else:
            self.transform_type = torchvision.transforms.Compose([transforms.Resize((224,224)),
                                                             transforms.ToTensor(),
                                                             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                                            ])
                
        
        
    def __getitem__(self, index):
        anchor_path = self.anchor_pathlist[index]
        positive_path = self.positive_pathlist[index]
        negative_path = self.negative_pathlist[index]
        anchor = self.transform_type(copy.deepcopy(Image.open(anchor_path[0])))
        positive = self.transform_type(copy.deepcopy(Image.open(positive_path[0])))
        negative = self.transform_type(copy.deepcopy(Image.open(negative_path[0])))  
        
        return anchor, positive, negative
    
    def __len__(self):
        
        return len(self.anchor_pathlist)
        

In [17]:
train_dataset = Mydata(trainlist_anchor, trainlist_positive, trainlist_negative, transform_type='train') 

In [18]:
train = DataLoader(train_dataset, batch_size=10, shuffle = True)

In [19]:
# for i, (anchor, positive, negative) in enumerate(train):
#     print(anchor[0])
len(train)

3000

Model

In [20]:
# task_model = models.resnet152(pretrained=True)
task_model = models.resnet101(pretrained=True)

for name, model in task_model.named_children():
    print(name)
    
for param in task_model.parameters():
    param.requires_grad = False

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [21]:
print(task_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
in_ftrs = task_model.fc.in_features
out_ftrs = task_model.fc.out_features

# update the fully connected layer
# task_model.fc = nn.Linear(in_ftrs, 128)
task_model.fc = torch.nn.Sequential(torch.nn.Linear(in_ftrs, 1024),
                                              torch.nn.ReLU(),
                                              torch.nn.BatchNorm1d(1024),
                                              torch.nn.Dropout(p=0.3),
                                              torch.nn.Linear(1024, 256))
print(task_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
for name, p in task_model.named_parameters():
    print(name, p.requires_grad)

conv1.weight False
bn1.weight False
bn1.bias False
layer1.0.conv1.weight False
layer1.0.bn1.weight False
layer1.0.bn1.bias False
layer1.0.conv2.weight False
layer1.0.bn2.weight False
layer1.0.bn2.bias False
layer1.0.conv3.weight False
layer1.0.bn3.weight False
layer1.0.bn3.bias False
layer1.0.downsample.0.weight False
layer1.0.downsample.1.weight False
layer1.0.downsample.1.bias False
layer1.1.conv1.weight False
layer1.1.bn1.weight False
layer1.1.bn1.bias False
layer1.1.conv2.weight False
layer1.1.bn2.weight False
layer1.1.bn2.bias False
layer1.1.conv3.weight False
layer1.1.bn3.weight False
layer1.1.bn3.bias False
layer1.2.conv1.weight False
layer1.2.bn1.weight False
layer1.2.bn1.bias False
layer1.2.conv2.weight False
layer1.2.bn2.weight False
layer1.2.bn2.bias False
layer1.2.conv3.weight False
layer1.2.bn3.weight False
layer1.2.bn3.bias False
layer2.0.conv1.weight False
layer2.0.bn1.weight False
layer2.0.bn1.bias False
layer2.0.conv2.weight False
layer2.0.bn2.weight False
layer2.0.bn2

In [24]:
# use cuda
task_model = task_model.to(device)

In [25]:
criterion = nn.TripletMarginLoss(margin=0.5, p=2, reduction='mean')
optimizer = optim.Adam(task_model.parameters(), lr = 0.000025)

In [27]:
epoches = 5
task_model.train()
scaler = GradScaler()
for epoch in range(epoches):
    train_loss = 0.0
    training_loss = []
    correct_all = 0
    train_all = 0
    for batch_id, (anchor, positive, negative) in enumerate(train):
        optimizer.zero_grad() 
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)
            
        anchor_feature = torch.nn.functional.normalize(task_model(anchor))
        positive_feature = torch.nn.functional.normalize(task_model(positive))
        negative_feature = torch.nn.functional.normalize(task_model(negative))
            
        with autocast():
            loss = criterion(anchor_feature, positive_feature, negative_feature)
        training_loss.append(loss.item())
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
#         loss.backward()
#         optimizer.step()
        
        correct = torch.sum(torch.sum(torch.square(anchor_feature - positive_feature), 1) < 
                            torch.sum(torch.square(anchor_feature - negative_feature), 1)).item()
        accuracy = correct / anchor_feature.size()[0]
        correct_all += correct
        train_all += anchor_feature.size()[0]
        
        print('Train epoch:{} ({:.1f}%) Loss:{:.6f}  Accuracy:{:.2f}%'.format((epoch+1), 
                                                            100.*(batch_id + 1)/len(train),
                                                           loss.item(),100*accuracy))
        
    training_loss = np.average(training_loss)
            
    print('Train epoch:{}\t Loss:{:.6f}, Accuracy:{:.2f}'.format(epoch+1, training_loss, 100*correct_all/train_all))

Train epoch:1 (0.0%) Loss:0.488179  Accuracy:50.00%
Train epoch:1 (0.1%) Loss:0.462017  Accuracy:50.00%
Train epoch:1 (0.1%) Loss:0.457715  Accuracy:50.00%
Train epoch:1 (0.1%) Loss:0.471507  Accuracy:60.00%
Train epoch:1 (0.2%) Loss:0.470360  Accuracy:40.00%
Train epoch:1 (0.2%) Loss:0.551315  Accuracy:40.00%
Train epoch:1 (0.2%) Loss:0.294769  Accuracy:70.00%
Train epoch:1 (0.3%) Loss:0.149313  Accuracy:100.00%
Train epoch:1 (0.3%) Loss:0.463754  Accuracy:40.00%
Train epoch:1 (0.3%) Loss:0.513598  Accuracy:60.00%
Train epoch:1 (0.4%) Loss:0.580158  Accuracy:60.00%
Train epoch:1 (0.4%) Loss:0.278529  Accuracy:80.00%
Train epoch:1 (0.4%) Loss:0.327293  Accuracy:70.00%
Train epoch:1 (0.5%) Loss:0.395966  Accuracy:80.00%
Train epoch:1 (0.5%) Loss:0.229917  Accuracy:80.00%
Train epoch:1 (0.5%) Loss:0.434052  Accuracy:50.00%
Train epoch:1 (0.6%) Loss:0.296496  Accuracy:70.00%
Train epoch:1 (0.6%) Loss:0.301103  Accuracy:80.00%
Train epoch:1 (0.6%) Loss:0.375051  Accuracy:70.00%
Train epoch

Train epoch:1 (5.3%) Loss:0.513451  Accuracy:50.00%
Train epoch:1 (5.3%) Loss:0.372232  Accuracy:70.00%
Train epoch:1 (5.4%) Loss:0.289690  Accuracy:80.00%
Train epoch:1 (5.4%) Loss:0.268120  Accuracy:80.00%
Train epoch:1 (5.4%) Loss:0.303921  Accuracy:70.00%
Train epoch:1 (5.5%) Loss:0.327225  Accuracy:70.00%
Train epoch:1 (5.5%) Loss:0.409530  Accuracy:80.00%
Train epoch:1 (5.5%) Loss:0.550365  Accuracy:40.00%
Train epoch:1 (5.6%) Loss:0.308672  Accuracy:70.00%
Train epoch:1 (5.6%) Loss:0.331275  Accuracy:80.00%
Train epoch:1 (5.6%) Loss:0.338977  Accuracy:70.00%
Train epoch:1 (5.7%) Loss:0.372265  Accuracy:70.00%
Train epoch:1 (5.7%) Loss:0.407325  Accuracy:60.00%
Train epoch:1 (5.7%) Loss:0.471521  Accuracy:60.00%
Train epoch:1 (5.8%) Loss:0.686142  Accuracy:30.00%
Train epoch:1 (5.8%) Loss:0.367115  Accuracy:80.00%
Train epoch:1 (5.8%) Loss:0.371517  Accuracy:80.00%
Train epoch:1 (5.9%) Loss:0.256127  Accuracy:80.00%
Train epoch:1 (5.9%) Loss:0.357367  Accuracy:70.00%
Train epoch:

Train epoch:1 (10.6%) Loss:0.455308  Accuracy:50.00%
Train epoch:1 (10.6%) Loss:0.305858  Accuracy:70.00%
Train epoch:1 (10.6%) Loss:0.326482  Accuracy:80.00%
Train epoch:1 (10.7%) Loss:0.368351  Accuracy:80.00%
Train epoch:1 (10.7%) Loss:0.366327  Accuracy:70.00%
Train epoch:1 (10.7%) Loss:0.319227  Accuracy:80.00%
Train epoch:1 (10.8%) Loss:0.390253  Accuracy:60.00%
Train epoch:1 (10.8%) Loss:0.276736  Accuracy:70.00%
Train epoch:1 (10.8%) Loss:0.418177  Accuracy:50.00%
Train epoch:1 (10.9%) Loss:0.578508  Accuracy:50.00%
Train epoch:1 (10.9%) Loss:0.532960  Accuracy:40.00%
Train epoch:1 (10.9%) Loss:0.167776  Accuracy:100.00%
Train epoch:1 (11.0%) Loss:0.331180  Accuracy:60.00%
Train epoch:1 (11.0%) Loss:0.473857  Accuracy:50.00%
Train epoch:1 (11.0%) Loss:0.223350  Accuracy:90.00%
Train epoch:1 (11.1%) Loss:0.302731  Accuracy:70.00%
Train epoch:1 (11.1%) Loss:0.362307  Accuracy:80.00%
Train epoch:1 (11.1%) Loss:0.339643  Accuracy:70.00%
Train epoch:1 (11.2%) Loss:0.577785  Accuracy

Train epoch:1 (15.7%) Loss:0.337511  Accuracy:70.00%
Train epoch:1 (15.8%) Loss:0.490022  Accuracy:60.00%
Train epoch:1 (15.8%) Loss:0.360010  Accuracy:80.00%
Train epoch:1 (15.8%) Loss:0.221552  Accuracy:80.00%
Train epoch:1 (15.9%) Loss:0.213586  Accuracy:80.00%
Train epoch:1 (15.9%) Loss:0.459708  Accuracy:60.00%
Train epoch:1 (15.9%) Loss:0.351126  Accuracy:70.00%
Train epoch:1 (16.0%) Loss:0.250167  Accuracy:80.00%
Train epoch:1 (16.0%) Loss:0.289976  Accuracy:70.00%
Train epoch:1 (16.0%) Loss:0.192331  Accuracy:80.00%
Train epoch:1 (16.1%) Loss:0.454428  Accuracy:70.00%
Train epoch:1 (16.1%) Loss:0.232475  Accuracy:70.00%
Train epoch:1 (16.1%) Loss:0.325050  Accuracy:90.00%
Train epoch:1 (16.2%) Loss:0.360975  Accuracy:60.00%
Train epoch:1 (16.2%) Loss:0.461902  Accuracy:50.00%
Train epoch:1 (16.2%) Loss:0.243467  Accuracy:90.00%
Train epoch:1 (16.3%) Loss:0.262258  Accuracy:80.00%
Train epoch:1 (16.3%) Loss:0.400026  Accuracy:50.00%
Train epoch:1 (16.3%) Loss:0.334298  Accuracy:

Train epoch:1 (20.9%) Loss:0.358737  Accuracy:60.00%
Train epoch:1 (20.9%) Loss:0.170926  Accuracy:90.00%
Train epoch:1 (21.0%) Loss:0.281155  Accuracy:80.00%
Train epoch:1 (21.0%) Loss:0.403253  Accuracy:50.00%
Train epoch:1 (21.0%) Loss:0.266363  Accuracy:70.00%
Train epoch:1 (21.1%) Loss:0.273062  Accuracy:60.00%
Train epoch:1 (21.1%) Loss:0.337962  Accuracy:70.00%
Train epoch:1 (21.1%) Loss:0.266442  Accuracy:80.00%
Train epoch:1 (21.2%) Loss:0.440856  Accuracy:60.00%
Train epoch:1 (21.2%) Loss:0.382722  Accuracy:70.00%
Train epoch:1 (21.2%) Loss:0.263569  Accuracy:90.00%
Train epoch:1 (21.3%) Loss:0.280618  Accuracy:90.00%
Train epoch:1 (21.3%) Loss:0.291113  Accuracy:70.00%
Train epoch:1 (21.3%) Loss:0.453857  Accuracy:50.00%
Train epoch:1 (21.4%) Loss:0.232816  Accuracy:80.00%
Train epoch:1 (21.4%) Loss:0.263410  Accuracy:70.00%
Train epoch:1 (21.4%) Loss:0.253358  Accuracy:70.00%
Train epoch:1 (21.5%) Loss:0.402212  Accuracy:60.00%
Train epoch:1 (21.5%) Loss:0.228333  Accuracy:

...

Train epoch:5 (76.2%) Loss:0.380690  Accuracy:70.00%
Train epoch:5 (76.3%) Loss:0.439210  Accuracy:60.00%
Train epoch:5 (76.3%) Loss:0.287462  Accuracy:80.00%
Train epoch:5 (76.3%) Loss:0.348159  Accuracy:70.00%
Train epoch:5 (76.4%) Loss:0.469153  Accuracy:50.00%
Train epoch:5 (76.4%) Loss:0.535999  Accuracy:40.00%
Train epoch:5 (76.4%) Loss:0.207459  Accuracy:80.00%
Train epoch:5 (76.5%) Loss:0.317594  Accuracy:70.00%
Train epoch:5 (76.5%) Loss:0.256279  Accuracy:90.00%
Train epoch:5 (76.5%) Loss:0.382475  Accuracy:80.00%
Train epoch:5 (76.6%) Loss:0.326179  Accuracy:60.00%
Train epoch:5 (76.6%) Loss:0.336517  Accuracy:70.00%
Train epoch:5 (76.6%) Loss:0.392396  Accuracy:70.00%
Train epoch:5 (76.7%) Loss:0.209256  Accuracy:90.00%
Train epoch:5 (76.7%) Loss:0.290436  Accuracy:80.00%
Train epoch:5 (76.7%) Loss:0.292596  Accuracy:80.00%
Train epoch:5 (76.8%) Loss:0.307473  Accuracy:90.00%
Train epoch:5 (76.8%) Loss:0.223578  Accuracy:90.00%
Train epoch:5 (76.8%) Loss:0.530793  Accuracy:

Train epoch:5 (81.4%) Loss:0.423563  Accuracy:50.00%
Train epoch:5 (81.4%) Loss:0.371623  Accuracy:70.00%
Train epoch:5 (81.5%) Loss:0.214616  Accuracy:80.00%
Train epoch:5 (81.5%) Loss:0.274476  Accuracy:80.00%
Train epoch:5 (81.5%) Loss:0.281041  Accuracy:80.00%
Train epoch:5 (81.6%) Loss:0.378938  Accuracy:70.00%
Train epoch:5 (81.6%) Loss:0.245189  Accuracy:70.00%
Train epoch:5 (81.6%) Loss:0.407127  Accuracy:60.00%
Train epoch:5 (81.7%) Loss:0.225953  Accuracy:90.00%
Train epoch:5 (81.7%) Loss:0.225282  Accuracy:90.00%
Train epoch:5 (81.7%) Loss:0.429185  Accuracy:70.00%
Train epoch:5 (81.8%) Loss:0.407911  Accuracy:90.00%
Train epoch:5 (81.8%) Loss:0.397973  Accuracy:50.00%
Train epoch:5 (81.8%) Loss:0.624381  Accuracy:50.00%
Train epoch:5 (81.9%) Loss:0.326913  Accuracy:80.00%
Train epoch:5 (81.9%) Loss:0.286356  Accuracy:70.00%
Train epoch:5 (81.9%) Loss:0.329255  Accuracy:60.00%
Train epoch:5 (82.0%) Loss:0.231681  Accuracy:80.00%
Train epoch:5 (82.0%) Loss:0.481524  Accuracy:

Train epoch:5 (86.6%) Loss:0.536146  Accuracy:60.00%
Train epoch:5 (86.6%) Loss:0.307189  Accuracy:80.00%
Train epoch:5 (86.6%) Loss:0.217116  Accuracy:90.00%
Train epoch:5 (86.7%) Loss:0.459817  Accuracy:70.00%
Train epoch:5 (86.7%) Loss:0.428147  Accuracy:60.00%
Train epoch:5 (86.7%) Loss:0.237079  Accuracy:70.00%
Train epoch:5 (86.8%) Loss:0.195574  Accuracy:100.00%
Train epoch:5 (86.8%) Loss:0.272315  Accuracy:80.00%
Train epoch:5 (86.8%) Loss:0.168923  Accuracy:80.00%
Train epoch:5 (86.9%) Loss:0.402852  Accuracy:80.00%
Train epoch:5 (86.9%) Loss:0.452564  Accuracy:60.00%
Train epoch:5 (86.9%) Loss:0.395023  Accuracy:60.00%
Train epoch:5 (87.0%) Loss:0.318503  Accuracy:80.00%
Train epoch:5 (87.0%) Loss:0.249350  Accuracy:70.00%
Train epoch:5 (87.0%) Loss:0.580320  Accuracy:60.00%
Train epoch:5 (87.1%) Loss:0.282307  Accuracy:80.00%
Train epoch:5 (87.1%) Loss:0.132941  Accuracy:90.00%
Train epoch:5 (87.1%) Loss:0.403936  Accuracy:60.00%
Train epoch:5 (87.2%) Loss:0.292049  Accuracy

Train epoch:5 (91.7%) Loss:0.226337  Accuracy:70.00%
Train epoch:5 (91.8%) Loss:0.359344  Accuracy:80.00%
Train epoch:5 (91.8%) Loss:0.515202  Accuracy:50.00%
Train epoch:5 (91.8%) Loss:0.413290  Accuracy:50.00%
Train epoch:5 (91.9%) Loss:0.435660  Accuracy:60.00%
Train epoch:5 (91.9%) Loss:0.336834  Accuracy:80.00%
Train epoch:5 (91.9%) Loss:0.289566  Accuracy:80.00%
Train epoch:5 (92.0%) Loss:0.382377  Accuracy:70.00%
Train epoch:5 (92.0%) Loss:0.615178  Accuracy:40.00%
Train epoch:5 (92.0%) Loss:0.358051  Accuracy:70.00%
Train epoch:5 (92.1%) Loss:0.319812  Accuracy:80.00%
Train epoch:5 (92.1%) Loss:0.483685  Accuracy:40.00%
Train epoch:5 (92.1%) Loss:0.451414  Accuracy:60.00%
Train epoch:5 (92.2%) Loss:0.324092  Accuracy:70.00%
Train epoch:5 (92.2%) Loss:0.459461  Accuracy:70.00%
Train epoch:5 (92.2%) Loss:0.560573  Accuracy:40.00%
Train epoch:5 (92.3%) Loss:0.245767  Accuracy:80.00%
Train epoch:5 (92.3%) Loss:0.313146  Accuracy:80.00%
Train epoch:5 (92.3%) Loss:0.305001  Accuracy:

Train epoch:5 (96.9%) Loss:0.318650  Accuracy:70.00%
Train epoch:5 (96.9%) Loss:0.254448  Accuracy:80.00%
Train epoch:5 (97.0%) Loss:0.284320  Accuracy:70.00%
Train epoch:5 (97.0%) Loss:0.501116  Accuracy:40.00%
Train epoch:5 (97.0%) Loss:0.394255  Accuracy:50.00%
Train epoch:5 (97.1%) Loss:0.534342  Accuracy:50.00%
Train epoch:5 (97.1%) Loss:0.156110  Accuracy:100.00%
Train epoch:5 (97.1%) Loss:0.329097  Accuracy:70.00%
Train epoch:5 (97.2%) Loss:0.473834  Accuracy:50.00%
Train epoch:5 (97.2%) Loss:0.443008  Accuracy:80.00%
Train epoch:5 (97.2%) Loss:0.295131  Accuracy:70.00%
Train epoch:5 (97.3%) Loss:0.168393  Accuracy:100.00%
Train epoch:5 (97.3%) Loss:0.402484  Accuracy:70.00%
Train epoch:5 (97.3%) Loss:0.254891  Accuracy:80.00%
Train epoch:5 (97.4%) Loss:0.242060  Accuracy:80.00%
Train epoch:5 (97.4%) Loss:0.485401  Accuracy:40.00%
Train epoch:5 (97.4%) Loss:0.387387  Accuracy:60.00%
Train epoch:5 (97.5%) Loss:0.522649  Accuracy:40.00%
Train epoch:5 (97.5%) Loss:0.143596  Accurac

Save Model

In [29]:
torch.save(task_model, 'task3_model_resnet101_b10_mhalf_normalize.pth')

Load Trained Model

In [18]:
task_model = torch.load( 'task3_model_resnet101_b10_m1/2_normalize.pth')
task_model = task_model.to(device)

In [64]:
print(task_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [65]:
valid_dataset = Mydata(validlist_anchor, validlist_positive, validlist_negative) 
valid = DataLoader(valid_dataset, batch_size=1, shuffle = None)
valid_criterion = nn.TripletMarginLoss(margin=0.0, p=2, reduction='mean')

In [66]:
with torch.no_grad():
    task_model.eval()

    id = 1.0
    correct = 0
    for anchor, positive, negative in valid:
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)
            
        anchor_feature = task_model(anchor)
        print(anchor_feature.shape)
        positive_feature = task_model(positive)
        negative_feature = task_model(negative)
        ap = torch.dist(anchor_feature, positive_feature, p=2)
        an = torch.dist(anchor_feature, negative_feature, p=2)
        loss = valid_criterion(anchor_feature, positive_feature, negative_feature)        
        

        if loss == 0.0:
            correct += 1
        print('Processing: {:.2f}%, ap:{:.6f}, an:{:.6f}, Loss:{:.6f}, Accuracy:{:.2f}%'.format(100*id/len(valid), ap, an, loss, 100*correct/id))
        
        id +=1
        
    print('Accuracy:{:.2f}%\t'.format(100*correct/len(valid)))

torch.Size([1, 128])
Processing: 0.00%, ap:6.609238, an:9.815788, Loss:0.000000, Accuracy:100.00%
torch.Size([1, 128])
Processing: 0.01%, ap:12.793303, an:7.619909, Loss:5.173393, Accuracy:50.00%
torch.Size([1, 128])
Processing: 0.01%, ap:8.006414, an:11.042895, Loss:0.000000, Accuracy:66.67%
torch.Size([1, 128])
Processing: 0.01%, ap:5.978226, an:8.829000, Loss:0.000000, Accuracy:75.00%
torch.Size([1, 128])
Processing: 0.02%, ap:11.588436, an:11.736182, Loss:0.000000, Accuracy:80.00%
torch.Size([1, 128])
Processing: 0.02%, ap:7.236660, an:9.463385, Loss:0.000000, Accuracy:83.33%
torch.Size([1, 128])
Processing: 0.02%, ap:8.403225, an:9.592742, Loss:0.000000, Accuracy:85.71%
torch.Size([1, 128])
Processing: 0.03%, ap:10.678773, an:8.576496, Loss:2.102279, Accuracy:75.00%
torch.Size([1, 128])
Processing: 0.03%, ap:7.602097, an:11.109422, Loss:0.000000, Accuracy:77.78%
torch.Size([1, 128])
Processing: 0.03%, ap:23.078255, an:23.003866, Loss:0.074389, Accuracy:70.00%
torch.Size([1, 128])


torch.Size([1, 128])
Processing: 0.30%, ap:6.060443, an:11.224395, Loss:0.000000, Accuracy:77.27%
torch.Size([1, 128])
Processing: 0.30%, ap:11.913447, an:10.606452, Loss:1.306996, Accuracy:76.40%
torch.Size([1, 128])
Processing: 0.30%, ap:6.712139, an:11.723096, Loss:0.000000, Accuracy:76.67%
torch.Size([1, 128])
Processing: 0.31%, ap:7.806514, an:12.303802, Loss:0.000000, Accuracy:76.92%
torch.Size([1, 128])
Processing: 0.31%, ap:9.304866, an:7.830993, Loss:1.473870, Accuracy:76.09%
torch.Size([1, 128])
Processing: 0.32%, ap:8.171826, an:8.681499, Loss:0.000000, Accuracy:76.34%
torch.Size([1, 128])
Processing: 0.32%, ap:7.523466, an:9.997160, Loss:0.000000, Accuracy:76.60%
torch.Size([1, 128])
Processing: 0.32%, ap:7.780415, an:10.007460, Loss:0.000000, Accuracy:76.84%
torch.Size([1, 128])
Processing: 0.33%, ap:7.501135, an:8.767707, Loss:0.000000, Accuracy:77.08%
torch.Size([1, 128])
Processing: 0.33%, ap:9.752672, an:10.233233, Loss:0.000000, Accuracy:77.32%
torch.Size([1, 128])
Pr

torch.Size([1, 128])
Processing: 0.59%, ap:7.341298, an:11.040212, Loss:0.000000, Accuracy:74.86%
torch.Size([1, 128])
Processing: 0.60%, ap:9.888515, an:9.810363, Loss:0.078151, Accuracy:74.43%
torch.Size([1, 128])
Processing: 0.60%, ap:7.054688, an:9.947088, Loss:0.000000, Accuracy:74.58%
torch.Size([1, 128])
Processing: 0.60%, ap:6.813385, an:15.280500, Loss:0.000000, Accuracy:74.72%
torch.Size([1, 128])
Processing: 0.61%, ap:8.024885, an:8.852912, Loss:0.000000, Accuracy:74.86%
torch.Size([1, 128])
Processing: 0.61%, ap:6.394685, an:9.384050, Loss:0.000000, Accuracy:75.00%
torch.Size([1, 128])
Processing: 0.61%, ap:9.287433, an:12.238974, Loss:0.000000, Accuracy:75.14%
torch.Size([1, 128])
Processing: 0.62%, ap:9.690494, an:12.545118, Loss:0.000000, Accuracy:75.27%
torch.Size([1, 128])
Processing: 0.62%, ap:8.000631, an:9.436887, Loss:0.000000, Accuracy:75.41%
torch.Size([1, 128])
Processing: 0.62%, ap:8.466611, an:8.968262, Loss:0.000000, Accuracy:75.54%
torch.Size([1, 128])
Proce

torch.Size([1, 128])
Processing: 0.88%, ap:7.479549, an:15.454004, Loss:0.000000, Accuracy:77.01%
torch.Size([1, 128])
Processing: 0.89%, ap:7.732955, an:7.934618, Loss:0.000000, Accuracy:77.10%
torch.Size([1, 128])
Processing: 0.89%, ap:10.011417, an:12.863280, Loss:0.000000, Accuracy:77.19%
torch.Size([1, 128])
Processing: 0.89%, ap:6.580761, an:7.532143, Loss:0.000000, Accuracy:77.27%
torch.Size([1, 128])
Processing: 0.90%, ap:8.257527, an:10.178003, Loss:0.000000, Accuracy:77.36%
torch.Size([1, 128])
Processing: 0.90%, ap:10.883465, an:12.648511, Loss:0.000000, Accuracy:77.44%
torch.Size([1, 128])
Processing: 0.90%, ap:10.428250, an:11.112697, Loss:0.000000, Accuracy:77.53%
torch.Size([1, 128])
Processing: 0.91%, ap:9.248777, an:12.196543, Loss:0.000000, Accuracy:77.61%
torch.Size([1, 128])
Processing: 0.91%, ap:5.701832, an:15.910764, Loss:0.000000, Accuracy:77.70%
torch.Size([1, 128])
Processing: 0.91%, ap:6.486227, an:14.059130, Loss:0.000000, Accuracy:77.78%
torch.Size([1, 128]

Processing: 1.17%, ap:7.504258, an:8.798330, Loss:0.000000, Accuracy:78.84%
torch.Size([1, 128])
Processing: 1.17%, ap:9.771715, an:10.265906, Loss:0.000000, Accuracy:78.90%
torch.Size([1, 128])
Processing: 1.18%, ap:7.520895, an:10.574330, Loss:0.000000, Accuracy:78.96%
torch.Size([1, 128])
Processing: 1.18%, ap:6.931899, an:9.715270, Loss:0.000000, Accuracy:79.02%
torch.Size([1, 128])
Processing: 1.18%, ap:9.133407, an:9.898877, Loss:0.000000, Accuracy:79.08%
torch.Size([1, 128])
Processing: 1.19%, ap:5.953062, an:22.757326, Loss:0.000000, Accuracy:79.14%
torch.Size([1, 128])
Processing: 1.19%, ap:11.407626, an:10.108097, Loss:1.299529, Accuracy:78.92%
torch.Size([1, 128])
Processing: 1.19%, ap:6.468446, an:13.426650, Loss:0.000000, Accuracy:78.98%
torch.Size([1, 128])
Processing: 1.20%, ap:8.326679, an:8.928882, Loss:0.000000, Accuracy:79.04%
torch.Size([1, 128])
Processing: 1.20%, ap:8.113533, an:10.194484, Loss:0.000000, Accuracy:79.10%
torch.Size([1, 128])
Processing: 1.20%, ap:6

...

torch.Size([1, 128])
Processing: 3.50%, ap:9.113538, an:11.360537, Loss:0.000000, Accuracy:78.78%
torch.Size([1, 128])
Processing: 3.50%, ap:7.453188, an:10.246001, Loss:0.000000, Accuracy:78.80%
torch.Size([1, 128])
Processing: 3.50%, ap:8.663367, an:11.995512, Loss:0.000000, Accuracy:78.82%
torch.Size([1, 128])
Processing: 3.51%, ap:7.352862, an:7.310209, Loss:0.042652, Accuracy:78.74%
torch.Size([1, 128])
Processing: 3.51%, ap:6.279077, an:7.846793, Loss:0.000000, Accuracy:78.76%
torch.Size([1, 128])
Processing: 3.51%, ap:8.311239, an:8.174679, Loss:0.136559, Accuracy:78.69%
torch.Size([1, 128])
Processing: 3.52%, ap:7.548255, an:6.746135, Loss:0.802119, Accuracy:78.61%
torch.Size([1, 128])
Processing: 3.52%, ap:7.056071, an:11.110219, Loss:0.000000, Accuracy:78.63%
torch.Size([1, 128])
Processing: 3.52%, ap:6.867854, an:8.857260, Loss:0.000000, Accuracy:78.65%
torch.Size([1, 128])
Processing: 3.53%, ap:7.806732, an:9.081096, Loss:0.000000, Accuracy:78.67%
torch.Size([1, 128])
Proce

Processing: 3.79%, ap:8.601974, an:9.311718, Loss:0.000000, Accuracy:79.07%
torch.Size([1, 128])
Processing: 3.79%, ap:11.285368, an:9.457573, Loss:1.827794, Accuracy:79.00%
torch.Size([1, 128])
Processing: 3.79%, ap:10.190179, an:9.295164, Loss:0.895015, Accuracy:78.93%
torch.Size([1, 128])
Processing: 3.80%, ap:10.990143, an:11.781941, Loss:0.000000, Accuracy:78.95%
torch.Size([1, 128])
Processing: 3.80%, ap:9.003047, an:13.510571, Loss:0.000000, Accuracy:78.97%
torch.Size([1, 128])
Processing: 3.80%, ap:6.633739, an:12.341600, Loss:0.000000, Accuracy:78.98%
torch.Size([1, 128])
Processing: 3.81%, ap:5.674487, an:7.609137, Loss:0.000000, Accuracy:79.00%
torch.Size([1, 128])
Processing: 3.81%, ap:7.664275, an:6.727647, Loss:0.936627, Accuracy:78.93%
torch.Size([1, 128])
Processing: 3.82%, ap:9.008375, an:6.961751, Loss:2.046624, Accuracy:78.86%
torch.Size([1, 128])
Processing: 3.82%, ap:6.987297, an:8.871889, Loss:0.000000, Accuracy:78.88%
torch.Size([1, 128])
Processing: 3.82%, ap:11

torch.Size([1, 128])
Processing: 4.08%, ap:7.407594, an:8.459396, Loss:0.000000, Accuracy:78.67%
torch.Size([1, 128])
Processing: 4.09%, ap:8.916994, an:7.820540, Loss:1.096454, Accuracy:78.61%
torch.Size([1, 128])
Processing: 4.09%, ap:8.228116, an:10.857316, Loss:0.000000, Accuracy:78.62%
torch.Size([1, 128])
Processing: 4.09%, ap:9.621158, an:8.943739, Loss:0.677419, Accuracy:78.56%
torch.Size([1, 128])
Processing: 4.10%, ap:6.988372, an:10.667140, Loss:0.000000, Accuracy:78.58%
torch.Size([1, 128])
Processing: 4.10%, ap:7.188138, an:10.120645, Loss:0.000000, Accuracy:78.60%
torch.Size([1, 128])
Processing: 4.10%, ap:9.026373, an:8.268908, Loss:0.757464, Accuracy:78.53%
torch.Size([1, 128])
Processing: 4.11%, ap:10.694082, an:10.303007, Loss:0.391075, Accuracy:78.47%
torch.Size([1, 128])
Processing: 4.11%, ap:6.505695, an:11.186110, Loss:0.000000, Accuracy:78.48%
torch.Size([1, 128])
Processing: 4.11%, ap:9.279683, an:7.731978, Loss:1.547706, Accuracy:78.42%
torch.Size([1, 128])
Pro

torch.Size([1, 128])
Processing: 4.37%, ap:11.434684, an:7.133213, Loss:4.301472, Accuracy:78.37%
torch.Size([1, 128])
Processing: 4.37%, ap:8.442786, an:9.527449, Loss:0.000000, Accuracy:78.39%
torch.Size([1, 128])
Processing: 4.38%, ap:7.866305, an:7.486083, Loss:0.380223, Accuracy:78.33%
torch.Size([1, 128])
Processing: 4.38%, ap:9.635847, an:13.835633, Loss:0.000000, Accuracy:78.34%
torch.Size([1, 128])
Processing: 4.38%, ap:7.399079, an:9.816832, Loss:0.000000, Accuracy:78.36%
torch.Size([1, 128])
Processing: 4.39%, ap:6.104024, an:7.763549, Loss:0.000000, Accuracy:78.38%
torch.Size([1, 128])
Processing: 4.39%, ap:7.873085, an:11.554093, Loss:0.000000, Accuracy:78.40%
torch.Size([1, 128])
Processing: 4.39%, ap:7.210610, an:8.126993, Loss:0.000000, Accuracy:78.41%
torch.Size([1, 128])
Processing: 4.40%, ap:9.423544, an:7.962797, Loss:1.460746, Accuracy:78.35%
torch.Size([1, 128])
Processing: 4.40%, ap:9.046731, an:9.090544, Loss:0.000000, Accuracy:78.37%
torch.Size([1, 128])
Proces

KeyboardInterrupt: 

In [23]:
valid_trail = DataLoader(valid_dataset, batch_size=10, shuffle = None)
valid_criterion_trail = nn.TripletMarginLoss(margin=0.0, p=2, reduction='none')

In [25]:
with torch.no_grad():
    task_model.eval()
    correct_all = 0
    valid_all = 0
    for batch_id, (anchor, positive, negative) in enumerate(valid_trail):
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)
            
        anchor_feature = torch.nn.functional.normalize(task_model(anchor))
#         print(anchor_feature.shape)
        positive_feature = torch.nn.functional.normalize(task_model(positive))
        negative_feature = torch.nn.functional.normalize(task_model(negative))
        loss = valid_criterion_trail(anchor_feature, positive_feature, negative_feature)
        
        print(loss)
        correct = torch.sum(torch.sum(torch.square(anchor_feature - positive_feature), 1) < 
                            torch.sum(torch.square(anchor_feature - negative_feature), 1)).item()
        accuracy = correct / anchor_feature.size()[0]
        correct_all += correct
        valid_all += anchor_feature.size()[0]
        
        print('Processing: {:.2f}%, Accuracy:{:.2f}%'.format(100*id/len(valid_trail), 100*accuracy))
        valid_loss.append(loss.item())
    
    print('Accuracy:{:.2f}%'.format(100*correct_all/valid_trail))

torch.Size([10, 128])
tensor([0.0000, 0.0000, 0.3886, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        2.1355], device='cuda:0')
torch.Size([10, 128])
tensor([2.0955, 0.2067, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.5512,
        0.0000], device='cuda:0')
torch.Size([10, 128])
tensor([0.0000, 0.0000, 0.1418, 0.0000, 2.1943, 0.0000, 0.8413, 0.6713, 0.9675,
        0.0000], device='cuda:0')
torch.Size([10, 128])
tensor([1.4604, 0.7073, 1.5947, 0.6721, 0.0000, 0.5373, 0.0000, 0.3448, 0.0000,
        0.0000], device='cuda:0')
torch.Size([10, 128])
tensor([0.0000, 0.2543, 0.0000, 0.0000, 0.6024, 0.8314, 0.1058, 0.0000, 0.0000,
        0.9370], device='cuda:0')
torch.Size([10, 128])
tensor([0.0000, 0.3830, 0.0000, 0.0000, 0.0000, 1.7679, 0.7162, 0.0000, 0.0000,
        0.0000], device='cuda:0')
torch.Size([10, 128])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3134, 0.0000, 0.0000, 0.0000,
        0.0000], device='cuda:0')
torch.Size([10, 128])
tensor([0.0000, 0.0000, 0.

KeyboardInterrupt: 

Get Test Dataset

In [30]:
test_triplets = open('test_triplets.txt','r')

In [32]:
testlist_anchor = []
testlist_positive = []
testlist_negative =[]
for line in test_triplets.readlines():
    image_id = line.strip().split(' ')
    testlist_anchor.append(['./food/'+str(image_id[0])+'.jpg','anchor'])
    testlist_positive.append(['./food/'+str(image_id[1])+'.jpg','positive'])
    testlist_negative.append(['./food/'+str(image_id[-1])+'.jpg','negative'])

In [33]:
len(testlist_anchor)

59544

In [34]:
testlist_anchor[0][0]

'./food/09896.jpg'

In [35]:
test_dataset = Mydata(testlist_anchor, testlist_positive, testlist_negative) 
test = DataLoader(test_dataset, batch_size=100, shuffle = None)

Test

In [36]:
test_criterion = nn.TripletMarginLoss(margin=0.0, p=2, reduction='none')

In [37]:
with torch.no_grad():
    task_model.eval()
    valid_loss = []
    avg_valid_loss = 0.0
    correct = 0
    id = 1.0
    output = []
    for batch_id, (anchor, positive, negative) in enumerate(test):
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)
            
        anchor_feature = torch.nn.functional.normalize(task_model(anchor))
        positive_feature = torch.nn.functional.normalize(task_model(positive))
        negative_feature = torch.nn.functional.normalize(task_model(negative))
#         loss = test_criterion(anchor_feature, positive_feature, negative_feature)
#         for i in loss:
#             if i == 0.0:
#                 output.append('1')
#             else:
#                 output.append('0')

        with autocast():        
            prediction = (torch.sum(torch.square(anchor_feature - positive_feature), 1) < 
                          torch.sum(torch.square(anchor_feature - negative_feature), 1)).tolist()
        
#         output.append(prediction)
        for i in prediction:
            if i == False:
                output.append('0')
            else:
                output.append('1')
        
        print('Processing: {:.2f}%'.format(100.*(batch_id+1)/len(test)))

Processing: 0.17%
Processing: 0.34%
Processing: 0.50%
Processing: 0.67%
Processing: 0.84%
Processing: 1.01%
Processing: 1.17%
Processing: 1.34%
Processing: 1.51%
Processing: 1.68%
Processing: 1.85%
Processing: 2.01%
Processing: 2.18%
Processing: 2.35%
Processing: 2.52%
Processing: 2.68%
Processing: 2.85%
Processing: 3.02%
Processing: 3.19%
Processing: 3.36%
Processing: 3.52%
Processing: 3.69%
Processing: 3.86%
Processing: 4.03%
Processing: 4.19%
Processing: 4.36%
Processing: 4.53%
Processing: 4.70%
Processing: 4.87%
Processing: 5.03%
Processing: 5.20%
Processing: 5.37%
Processing: 5.54%
Processing: 5.70%
Processing: 5.87%
Processing: 6.04%
Processing: 6.21%
Processing: 6.38%
Processing: 6.54%
Processing: 6.71%
Processing: 6.88%
Processing: 7.05%
Processing: 7.21%
Processing: 7.38%
Processing: 7.55%
Processing: 7.72%
Processing: 7.89%
Processing: 8.05%
Processing: 8.22%
Processing: 8.39%
Processing: 8.56%
Processing: 8.72%
Processing: 8.89%
Processing: 9.06%
Processing: 9.23%
Processing

Processing: 73.15%
Processing: 73.32%
Processing: 73.49%
Processing: 73.66%
Processing: 73.83%
Processing: 73.99%
Processing: 74.16%
Processing: 74.33%
Processing: 74.50%
Processing: 74.66%
Processing: 74.83%
Processing: 75.00%
Processing: 75.17%
Processing: 75.34%
Processing: 75.50%
Processing: 75.67%
Processing: 75.84%
Processing: 76.01%
Processing: 76.17%
Processing: 76.34%
Processing: 76.51%
Processing: 76.68%
Processing: 76.85%
Processing: 77.01%
Processing: 77.18%
Processing: 77.35%
Processing: 77.52%
Processing: 77.68%
Processing: 77.85%
Processing: 78.02%
Processing: 78.19%
Processing: 78.36%
Processing: 78.52%
Processing: 78.69%
Processing: 78.86%
Processing: 79.03%
Processing: 79.19%
Processing: 79.36%
Processing: 79.53%
Processing: 79.70%
Processing: 79.87%
Processing: 80.03%
Processing: 80.20%
Processing: 80.37%
Processing: 80.54%
Processing: 80.70%
Processing: 80.87%
Processing: 81.04%
Processing: 81.21%
Processing: 81.38%
Processing: 81.54%
Processing: 81.71%
Processing: 

Create result.txt

In [38]:
a = '\n'
result = open('result.txt','w')
result.write(a.join(output))
result.close()